In [1]:
import os
import re
import numpy as np
from tqdm import tqdm
import json
import copy

import tensorflow as tf
from transformers import *

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# 시각화

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string])
    plt.show()

In [3]:
#random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

BATCH_SIZE = 32
NUM_EPOCHS = 3
MAX_LEN = 111 # EDA에서 추출된 Max Length

In [4]:
data_path = 'C:/nlp/BERT/data_in/KOR/NER/'
op = "C:/nlp/BERT/data_out/"

train_path = os.path.join(data_path, "train.tsv")
test_path = os.path.join(data_path, "test.tsv")
label_path = os.path.join(data_path, "label.txt")

문장 : 금석객잔 여러분 감사드립니다. \n

띄어쓰기 분해: '금석객잔', '여러분', '감사드립니다.'

띄어쓰기 라벨 분해: 'ORG-B', 'O','O','O'

버트 토크나이저 분해: '금', '##석', '##객', ... 한 글자씩

라벨 데이터는 현재 띄어쓰기를 기분으로 분해에 맞춰져 있음. 버트 토크나이저를 사용하면 길이가 12인 토큰으로 분해.
그렇기에 라벨 데이터에 맞게 수정하는 작업이 필요로함.

In [5]:

def read_file(input_path):
    """Read tsv file, and return words and label as list"""
    with open(input_path, "r", encoding="utf-8") as f:
        sentences = []
        labels = []
        for line in f:
            split_line = line.strip().split("\t")
            sentences.append(split_line[0])
            labels.append(split_line[1])
        return sentences, labels

train_sentences, train_labels = read_file(train_path)

train_ner_dict = {"sentence": train_sentences, "label": train_labels}
train_ner_df = pd.DataFrame(train_ner_dict)

test_sentences, test_labels = read_file(test_path)
test_ner_dict = {"sentence": test_sentences, "label": test_labels}
test_ner_df = pd.DataFrame(test_ner_dict)

print(f"개체명 인식 학습 데이터 개수: {len(train_ner_df)}")
print(f"개체명 인식 테스트 데이터 개수: {len(test_ner_df)}")


개체명 인식 학습 데이터 개수: 81000
개체명 인식 테스트 데이터 개수: 9000


In [6]:
# Label 불러오기

def get_labels(label_path):
    return [label.strip() for label in open(os.path.join(label_path), 'r', encoding='utf-8')]

ner_labels = get_labels(label_path)

print(f"개체명 인식 레이블 개수: {len(ner_labels)}")

개체명 인식 레이블 개수: 30


In [9]:
train_ner_df.head(10)

,sentence,label
0,"금석객잔 여러분, 감사드립니다 .",ORG-B O O O
1,이기범 한두 쪽을 먹고 10분 후쯤 화제인을 먹는 것이 좋다고 한다 .,PER-B O O O TIM-B TIM-I CVL-B O O O O O
2,7-8위 결정전에서 김중배 무스파타(샌안토니오)가 참은 법국을 누르고 유럽축구선수권...,EVT-B EVT-I PER-B PER-I O LOC-B O EVT-B CVL-B O O
3,스코틀랜드의 한 마을에서 보통하게 살고 있다는 이 기혼 남성의 시조가 유튜브 등에서...,LOC-B NUM-B NUM-I O O O O O O O O O O O O O CV...
4,보니까 저 옆에 사조가 있어요 .,O O O O O O
5,24회 최경운호의 좌익선상 28루타로 포문을 연 한화는 후속 서동원이 적시타를 날려...,NUM-B PER-B O NUM-B O O ORG-B O PER-B O O O O O
6,바둑선수가 묘하게 닮았는데요 .,CVL-B O O O
7,▲ '新플레이메이커' NO.7 박하성 - 1경기 30골30도움공수 운동경기가 풀리지...,O CVL-B NUM-B PER-B O NUM-B NUM-B O O O O O O ...
8,우려는 비현실이 됐다 .,O O O O
9,(이석무 귀재 smlee@mydaily.co.kr),PER-B CVL-B TRM-B


In [10]:
test_ner_df.head(10)

,sentence,label
0,나아가 한 스트로크를 하는 제한시간에 대한 지침도 있다 .,O NUM-B NUM-I O O O O O O
1,그건 관두 영업적 득공을 버리고 퇴은을 원하는 엘레나 대행 궐녀를 닮은 여인들을 교...,O O O O O O O PER-B O O O CVL-B O O O
2,송인영은 5년 근화향에서 개최된 CJ나인브릿지클래식 토요돌봄으로 'LPGA 직행티킷...,PER-B DAT-B LOC-B O EVT-B CVL-B ORG-B O O CVL-B O
3,역전패' 단국 월드리그 7연패…장영달회장 인퇴,O EVT-B EVT-I NUM-B O
4,‘국내 최대 통신그룹인 KT호의 선장은 누가 될까 . ',O O O ORG-B CVL-B O O O O
5,해자부래기 등을 연출한 무하마드 무설탕껌의 신작인 이 소작은 특유의 희극적한 배역으...,O O O PER-B CVL-B O O O O O O O PER-B O O O O ...
6,"다들 보면, 보지는 못하더라도 안부전화 하고 에베레스트에서 한류스타들이라고, 잘나가...",O O O O O O LOC-B CVL-B O O O O O O O
7,"-5명의 해외 익명 부녀회원이 합동 연출, 에로스라는 분수로 7가지의 일담을 담아낸...",NUM-B O O CVL-B O O PER-B O NUM-B O O FLD-B O O
8,칼럼의 제호를 ‘믹스트존’이 아니라 ‘철망 밖’이라고 해야겠다 .,O O O O O O O O
9,한편 물음이 있다면 공중이 필수하다는 거예요 .,NUM-B O O O O O O


In [11]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", cache_dir='bert_ckpt')

pad_token_id = tokenizer.pad_token_id # 0
pad_token_label_id = 0
cls_token_label_id = 0
sep_token_label_id = 0

bert_ckpt에 버트 토크나이저를 불러와서 저장.

pad_token_id와 pad_token_label_id를 각각 0으로 지정. pad_token_id는 라벨이 시퀀스 길이에 맞춰져 있기 때문에 주어진 입력의 길이 기반으로 패딩하기 위해 필요한 값.

pad_token_label_id는 학습 시 라벨된 값 외에 학습에 영향을 미치지 않기 위해 설정되는 값이다.
추가로 cls와 sep도 pad와 같은 특정 값인 0을 할당. 이는 향후 loss를 구성할 때 시퀀스가 모두 영향을 미치는 개체명 분야의 특성 때문에 인식에 필요한 값 외에는 모두 0으로 라벨을 지정해 학습에 no 영향.

In [12]:
def bert_tokenizer(sent, MAX_LEN):
    
    encoded_dict = tokenizer.encode_plus(
        text = sent,
        truncation=True,
        add_special_tokens = True, 
        max_length = MAX_LEN,           
        pad_to_max_length = True,
        return_attention_mask = True  
    )
    
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask'] 
    token_type_id = encoded_dict['token_type_ids']
    
    return input_id, attention_mask, token_type_id

띄어쓰기 단위로 맞게 구성돼 있는 라벨을 버트 토크나이저에 맞는 형태로 변형.
라벨의 길이를 맞추는 방식에는 다양한 방식이 존재. 그 중에서 버트 토크나이저로 분해된 개체명의 첫 버트 토큰 부분을 시작을 상징하는 B(begin)라벨로 지정하고, 나머지 부분은 내부를 사징하는 I(inner)로 지정하는 방식을 사용할 예정.

In [13]:
def convert_label(words, labels_idx, ner_begin_label, max_seq_len):
            
    tokens = []
    label_ids = []

    for word, slot_label in zip(words, labels_idx):

        word_tokens = tokenizer.tokenize(word)
        if not word_tokens:
            word_tokens = [unk_token]
        tokens.extend(word_tokens)
        
        # 슬롯 레이블 값이 Begin이면 I로 추가
        if int(slot_label) in ner_begin_label:
            label_ids.extend([int(slot_label)] + [int(slot_label) + 1] * (len(word_tokens) - 1))
        else:
            label_ids.extend([int(slot_label)] * len(word_tokens))
  
    # [CLS] and [SEP] 설정
    special_tokens_count = 2
    if len(label_ids) > max_seq_len - special_tokens_count:
        label_ids = label_ids[: (max_seq_len - special_tokens_count)]

    # [SEP] 토큰 추가
    label_ids += [sep_token_label_id]

    # [CLS] 토큰 추가
    label_ids = [cls_token_label_id] + label_ids
    
    padding_length = max_seq_len - len(label_ids)
    label_ids = label_ids + ([pad_token_label_id] * padding_length)
    
    return label_ids

In [14]:
# test

ner_begin_label = [ner_labels.index(begin_label) for begin_label in ner_labels if "B" in begin_label]
ner_begin_label_string = [ner_labels[label_index] for label_index in ner_begin_label]

print(ner_begin_label)
print(ner_begin_label_string)

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]
['PER-B', 'FLD-B', 'AFW-B', 'ORG-B', 'LOC-B', 'CVL-B', 'DAT-B', 'TIM-B', 'NUM-B', 'EVT-B', 'ANM-B', 'PLT-B', 'MAT-B', 'TRM-B']


In [18]:

def create_inputs_targets(df):
    input_ids = []
    attention_masks = []
    token_type_ids = []
    label_list = []

    for i, data in enumerate(df[['sentence', 'label']].values):
        sentence, labels = data
        words = sentence.split()
        labels = labels.split()
        
        labels_idx = []
        
        for label in labels:
            labels_idx.append(ner_labels.index(label) if label in ner_labels else ner_labels.index("UNK"))

        assert len(words) == len(labels_idx)

        input_id, attention_mask, token_type_id = bert_tokenizer(sentence, MAX_LEN)

        convert_label_id = convert_label(words, labels_idx, ner_begin_label, MAX_LEN)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label_list.append(convert_label_id)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    label_list = np.asarray(label_list, dtype=int) #레이블 토크나이징 리스트
    inputs = (input_ids, attention_masks, token_type_ids)
    
    return inputs, label_list

train_inputs, train_labels = create_inputs_targets(train_ner_df)
test_inputs, test_labels = create_inputs_targets(test_ner_df)

In [19]:
class TFBertNERClassifier(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBertNERClassifier, self).__init__()

        self.bert = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.dropout = tf.keras.layers.Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(num_class, 
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range),
                                                name="ner_classifier")

    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):

        #outputs 값: # sequence_output, pooled_output, (hidden_states), (attentions)
        outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        sequence_output = outputs[0]
                
        sequence_output = self.dropout(sequence_output, training=training)
        logits = self.classifier(sequence_output)
        

        return logits

In [20]:
ner_model = TFBertNERClassifier(model_name='bert-base-multilingual-cased',
                                  dir_path='bert_ckpt',
                                  num_class=len(ner_labels))

In [21]:
def compute_loss(labels, logits):
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction=tf.keras.losses.Reduction.NONE
    )

    # 0의 레이블 값은 손실 값을 계산할 때 제외
    active_loss = tf.reshape(labels, (-1,)) != 0
        
    reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
        
    labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)
    
    return loss_fn(labels, reduced_logits)

In [22]:
class F1Metrics(tf.keras.callbacks.Callback):
    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def compute_f1_pre_rec(self, labels, preds):

        return {
            "precision": precision_score(labels, preds, suffix=True),
            "recall": recall_score(labels, preds, suffix=True),
            "f1": f1_score(labels, preds, suffix=True)
        }


    def show_report(self, labels, preds):
        return classification_report(labels, preds, suffix=True)
        
    def on_epoch_end(self, epoch, logs=None):

        results = {}
        
        pred = self.model.predict(self.x_eval)
        label = self.y_eval
        pred_argmax = np.argmax(pred, axis = 2)

        slot_label_map = {i: label for i, label in enumerate(ner_labels)}

        out_label_list = [[] for _ in range(label.shape[0])]
        preds_list = [[] for _ in range(label.shape[0])]

        for i in range(label.shape[0]):
            for j in range(label.shape[1]):
                if label[i, j] != 0:
                    out_label_list[i].append(slot_label_map[label[i][j]])
                    preds_list[i].append(slot_label_map[pred_argmax[i][j]])
                    
        result = self.compute_f1_pre_rec(out_label_list, preds_list)
        results.update(result)

        print("********")
        print("F1 Score")
        for key in sorted(results.keys()):
            print("{}, {:.4f}".format(key, results[key]))
        print("\n" + self.show_report(out_label_list, preds_list))
        print("********")

f1_score_callback = F1Metrics(test_inputs, test_labels)

In [23]:
# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule
optimizer = tf.keras.optimizers.Adam(3e-5)
# ner_model.compile(optimizer=optimizer, loss=compute_loss, run_eagerly=True)
ner_model.compile(optimizer=optimizer, loss=compute_loss)

In [24]:
model_name = "tf2_bert_ner"

checkpoint_path = os.path.join(op, model_name, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    
cp_callback = ModelCheckpoint(
    checkpoint_path, verbose=1, save_best_only=True, save_weights_only=True)

history = ner_model.fit(train_inputs, train_labels, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
                        callbacks=[cp_callback, f1_score_callback])

print(history.history)

C:/nlp/BERT/data_out/tf2_bert_ner -- Folder create complete 

Epoch 1/3


KeyboardInterrupt: 

In [ ]:
plot_graphs(history, 'loss')